In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import asyncio
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

In [2]:
model_client = OpenAIChatCompletionClient(
    model = "gemini-1.5-flash-8b",
    api_key = api_key,
)

In [4]:
narrator = AssistantAgent(
    name="narrator",
    model_client=model_client,
)

hero = AssistantAgent(
    name="hero",
    model_client=model_client,
)

guide = AssistantAgent(
    name="guide",
    model_client=model_client,
)

### FeedBack in Max Turns 

In [6]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants = [narrator, hero, guide],
    max_turns = 1,
)

In [13]:
from autogen_agentchat.ui import Console
async def main():
    task = "Write a story about a hero and his passion less than 30 words."
    
    while True:
        stream = team.run_stream(task=task)
        await Console(stream)

        feedback = input('Provide feedback to the hero(type "exit" to stop): ')

        if feedback.lower().strip() == "exit":
            break

        task = f"Revise the story based on the feedback: {feedback}"

In [14]:
await main()

---------- TextMessage (user) ----------
Write a story about a hero and his passion less than 30 words.
---------- TextMessage (narrator) ----------
Kael, a blacksmith, found his passion forging hope.  A fallen star's fire fueled a weapon—a world-altering choice.

---------- TextMessage (user) ----------
Revise the story based on the feedback: add to above that one day he a metroite fall from sky
---------- TextMessage (hero) ----------
One day, a meteor fell.  Kael, a blacksmith, found his passion forging hope from its fiery remnants—a world-altering choice.

---------- TextMessage (user) ----------
Revise the story based on the feedback: then he came it can be a ore that can used to create weapons
---------- TextMessage (guide) ----------
From a fallen meteor's fiery core, Kael, a blacksmith, found his passion: forging hope—a weapon of world-altering power.

---------- TextMessage (user) ----------
Revise the story based on the feedback: and he created a weapon secretly thta could de

### FeedBack in Termination Conditions

In [26]:
lazy_agent = AssistantAgent(
    name="lazy_agent",
    model_client=model_client,
    system_message='Give the answer of question asked. If you cannot complete the task, or not have any information then just say "TERMINATE" not any more than that.',
)

In [27]:
from autogen_agentchat.conditions import TextMentionTermination

text_termination = TextMentionTermination('TERMINATE')

In [28]:
lazy_team = RoundRobinGroupChat(
    participants = [lazy_agent],
    termination_condition = text_termination,
    max_turns = 2,
)

In [29]:
async def main():
    task = "Give me the Current Weather in San Francisco"

    while True:
        stream = lazy_team.run_stream(task=task)
        await Console(stream)

        feedback = input('Provide your FeedBack(type "exit" to stop): ')

        if feedback.lower().strip() == "exit":
            break

        task = feedback

In [31]:
await main()

---------- TextMessage (user) ----------
Give me the Current Weather in San Francisco
---------- TextMessage (lazy_agent) ----------
I do not have real-time access to weather information.  TERMINATE

---------- TextMessage (user) ----------
its sunny man
---------- TextMessage (lazy_agent) ----------
Okay.

---------- TextMessage (lazy_agent) ----------

---------- TextMessage (user) ----------
then what is the current weather in india
---------- TextMessage (lazy_agent) ----------
I do not have real-time access to weather information.  TERMINATE

---------- TextMessage (user) ----------
its also sunny man
---------- TextMessage (lazy_agent) ----------
Okay.

---------- TextMessage (lazy_agent) ----------

---------- TextMessage (user) ----------
the what is an Ai? provide answer and say exit
---------- TextMessage (lazy_agent) ----------
AI, or Artificial Intelligence, is a broad field encompassing computer systems designed to perform tasks that typically require human intelligence.  

### HandOff Termination

In [47]:
from autogen_agentchat.base import Handoff
agent = AssistantAgent(
    name="agent",
    model_client=model_client,
    handoffs=[Handoff(target="user", message='Transfering to user')],
    system_message='You are an agent that can answer questions. If you cannot answer the question or doesn.t have any information abot that, transfer it to user. When done, say "TERMINATE".',
)

In [48]:
from autogen_agentchat.conditions import HandoffTermination 

handoff_termination = HandoffTermination('user')
text_termination = TextMentionTermination('TERMINATE')

termination_condition = handoff_termination | text_termination

In [49]:
team = RoundRobinGroupChat(
    participants = [agent],
    termination_condition=termination_condition,
)

In [50]:
async def main():
    task = "Give me the current weather of New York."

    await Console(team.run_stream(task=task), output_stats=True)

In [51]:
await main()

---------- TextMessage (user) ----------
Give me the current weather of New York.
---------- ToolCallRequestEvent (agent) ----------
[FunctionCall(id='', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 58, Completion tokens: 5]
---------- ToolCallExecutionEvent (agent) ----------
[FunctionExecutionResult(content='Transfering to user', name='transfer_to_user', call_id='', is_error=False)]
---------- HandoffMessage (agent) ----------
Transfering to user
---------- Summary ----------
Number of messages: 4
Finish reason: Handoff to user from agent detected.
Total prompt tokens: 58
Total completion tokens: 5
Duration: 0.94 seconds


In [52]:
async def main():
    task = "Give me the current weather of New York."

    await Console(team.run_stream(task=task), output_stats=True)

    feedback = "The weather is Sunny ☀️"

    await Console(team.run_stream(task=feedback))

In [53]:
await main()

---------- TextMessage (user) ----------
Give me the current weather of New York.
---------- TextMessage (agent) ----------
I do not have access to real-time weather data.  To get the current weather in New York, please use a weather website or app.
TERMINATE

[Prompt tokens: 82, Completion tokens: 34]
---------- Summary ----------
Number of messages: 2
Finish reason: Text 'TERMINATE' mentioned
Total prompt tokens: 82
Total completion tokens: 34
Duration: 0.76 seconds
---------- TextMessage (user) ----------
The weather is Sunny ☀️
---------- TextMessage (agent) ----------
Okay.  Is there anything else I can help you with?

---------- TextMessage (agent) ----------

---------- TextMessage (agent) ----------
No, thank you.

---------- TextMessage (agent) ----------

---------- TextMessage (agent) ----------
TERMINATE

